In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
pd.options.mode.chained_assignment = None
from tqdm import tqdm
from imblearn import under_sampling

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
validation_data = pd.read_csv('/content/drive/MyDrive/IRDM Coursework 2/validation_data_cosine.csv',index_col=0)

In [4]:
final_train = pd.read_csv('/content/drive/MyDrive/IRDM Coursework 2/train_data_cosine.csv',index_col=0)

In [5]:
final_train['DocLen'] = 0
final_train['queryLen'] = 0
for i, row in final_train.iterrows():
  final_train['DocLen'][i] = len(final_train['passage'][i])
  final_train['queryLen'][i] = len(final_train['queries'][i])

In [6]:
validation_data['DocLen'] = 0
validation_data['queryLen'] = 0
for i, row in validation_data.iterrows():
  validation_data['DocLen'][i] = len(validation_data['passage'][i])
  validation_data['queryLen'][i] = len(validation_data['queries'][i])

In [7]:
t = under_sampling.RandomUnderSampler(random_state=42)

In [8]:
x_train = final_train[['cosine_similarity', 'DocLen', 'queryLen']].values
y_train = final_train[['relevancy']].values
x_train,y_train = t.fit_resample(x_train,y_train)

In [9]:
x_test = validation_data[['cosine_similarity', 'DocLen', 'queryLen']].values
y_test = validation_data[['relevancy']].values

In [10]:
# from sklearn.model_selection import train_test_split
# x_train,x_val,y_train,y_val=train_test_split(x,Y,test_size=0.10,shuffle=True,stratify = Y)

In [11]:
from sklearn.preprocessing import StandardScaler
##Normalization is done so that the difference between highest and lowest data point is not too large
import numpy as np
scaler=StandardScaler()
##To find mean and std dev
scaler.fit(x_train)
## Please Note - We are using mean and std dev of training data for testing data too. It is done to ensure that no information of test data is leaked to the model.
##Converting data into form where mean of data is 0 and std dev is 1 
X_train=scaler.transform(x_train)
X_test=scaler.transform(x_test)


In [13]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(20, input_dim=X_train.shape[1], activation='relu'))
##Dropout is used to avoid overfitting
keras.layers.Dropout(0.2)
model.add(Dense(20, activation='relu'))
keras.layers.Dropout(0.2)
model.add(Dense(20, activation='relu'))
keras.layers.Dropout(0.2)
##For classification problems, we usually use softmax as activation function in final layer
model.add(Dense(1, activation='sigmoid')) 
##Compiling the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.Precision()]) 

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 20)                80        
                                                                 
 dense_5 (Dense)             (None, 20)                420       
                                                                 
 dense_6 (Dense)             (None, 20)                420       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 941
Trainable params: 941
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_train, y_train, epochs=8, batch_size=80, validation_split=0.1)

Epoch 1/8
104/104 [==============================] - 2s 5ms/step - loss: 1.5976 - precision: 0.4477 - val_loss: 0.7266 - val_precision: 1.0000
Epoch 2/8
104/104 [==============================] - 0s 2ms/step - loss: 0.6900 - precision: 0.4907 - val_loss: 0.5649 - val_precision: 1.0000
Epoch 3/8
104/104 [==============================] - 0s 2ms/step - loss: 0.6924 - precision: 0.4605 - val_loss: 0.6582 - val_precision: 1.0000
Epoch 4/8
104/104 [==============================] - 0s 2ms/step - loss: 0.7075 - precision: 0.4624 - val_loss: 0.8131 - val_precision: 1.0000
Epoch 5/8
104/104 [==============================] - 0s 2ms/step - loss: 0.6993 - precision: 0.4445 - val_loss: 0.6558 - val_precision: 1.0000
Epoch 6/8
104/104 [==============================] - 0s 2ms/step - loss: 0.7014 - precision: 0.4594 - val_loss: 1.0226 - val_precision: 0.0000e+00
Epoch 7/8
104/104 [==============================] - 0s 2ms/step - loss: 0.7150 - precision: 0.4504 - val_loss: 0.8051 - val_precision: 1.

In [16]:
testing=model.evaluate(x_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1]+'uracy of Model on testing data', testing[1]*100))

287/287 [==============================] - 0s 2ms/step - loss: 0.6907 - precision: 0.6565

precisionuracy of Model on testing data: 65.65%


In [17]:
from sklearn.metrics import classification_report, confusion_matrix  
predictions = model.predict(X_test)

In [18]:
validation_data['score'] = predictions
validation_data.head()

,qid,pid,queries,passage,relevancy,passage_tokens,query_tokens,passage_embedding,query_embedding,cosine_similarity,DocLen,queryLen,score
0,1082792,1000084,what does the golgi apparatus do to the protei...,"Start studying Bonding, Carbs, Proteins, Lipid...",0.0,"['start', 'studying', 'bonding', 'carbs', 'pro...","['golgi', 'apparatus', 'proteins', 'lipids', '...",[ 0.2690694 0.07032246 -0.22095296 -0.134478...,[ 0.832084 -0.16500281 0.050816 0.16417 ...,0.617340,129,78,0.530650
1,995825,1000492,where is the graphic card located in the cpu,"For example, a “PC Expansion Card” maybe the j...",0.0,"['example', 'pc', 'expansion', 'card', 'maybe'...","['graphic', 'card', 'located', 'cpu']",[ 1.18339844e-01 1.92679703e-01 4.75641042e-...,[-0.01731001 0.32846758 0.68064654 0.689135...,0.831845,332,44,0.521313
2,995825,1000494,where is the graphic card located in the cpu,The Common Cards & Buses. The most common type...,0.0,"['common', 'cards', 'buses', 'common', 'types'...","['graphic', 'card', 'located', 'cpu']",[ 0.02057143 0.11528355 0.52425003 0.421483...,[-0.01731001 0.32846758 0.68064654 0.689135...,0.847569,338,44,0.525208
3,1091246,1000522,property premises meaning,The occurrence of since tells us that the firs...,0.0,"['occurrence', 'since', 'tells', 'us', 'first'...","['property', 'premises', 'meaning']",[ 0.24477586 0.12094858 -0.1262933 -0.035436...,[ 0.5320867 0.13899668 -0.18879335 -0.206136...,0.751343,304,25,0.492774
4,1047854,1000585,what is printing mechanism,Windows desktop applications Develop Desktop t...,0.0,"['windows', 'desktop', 'applications', 'develo...","['printing', 'mechanism']",[ 0.21993366 -0.27575365 0.6672145 0.224379...,[ 0.17438498 -0.67538 -0.050255 -0.132791...,0.813332,820,26,0.510789


In [19]:
validation_data['NN_rank'] = validation_data.groupby('qid')['score'].rank(method='first',ascending=False).astype('int')

In [20]:
validation_data.head()

,qid,pid,queries,passage,relevancy,passage_tokens,query_tokens,passage_embedding,query_embedding,cosine_similarity,DocLen,queryLen,score,NN_rank
0,1082792,1000084,what does the golgi apparatus do to the protei...,"Start studying Bonding, Carbs, Proteins, Lipid...",0.0,"['start', 'studying', 'bonding', 'carbs', 'pro...","['golgi', 'apparatus', 'proteins', 'lipids', '...",[ 0.2690694 0.07032246 -0.22095296 -0.134478...,[ 0.832084 -0.16500281 0.050816 0.16417 ...,0.617340,129,78,0.530650,449
1,995825,1000492,where is the graphic card located in the cpu,"For example, a “PC Expansion Card” maybe the j...",0.0,"['example', 'pc', 'expansion', 'card', 'maybe'...","['graphic', 'card', 'located', 'cpu']",[ 1.18339844e-01 1.92679703e-01 4.75641042e-...,[-0.01731001 0.32846758 0.68064654 0.689135...,0.831845,332,44,0.521313,241
2,995825,1000494,where is the graphic card located in the cpu,The Common Cards & Buses. The most common type...,0.0,"['common', 'cards', 'buses', 'common', 'types'...","['graphic', 'card', 'located', 'cpu']",[ 0.02057143 0.11528355 0.52425003 0.421483...,[-0.01731001 0.32846758 0.68064654 0.689135...,0.847569,338,44,0.525208,169
3,1091246,1000522,property premises meaning,The occurrence of since tells us that the firs...,0.0,"['occurrence', 'since', 'tells', 'us', 'first'...","['property', 'premises', 'meaning']",[ 0.24477586 0.12094858 -0.1262933 -0.035436...,[ 0.5320867 0.13899668 -0.18879335 -0.206136...,0.751343,304,25,0.492774,690
4,1047854,1000585,what is printing mechanism,Windows desktop applications Develop Desktop t...,0.0,"['windows', 'desktop', 'applications', 'develo...","['printing', 'mechanism']",[ 0.21993366 -0.27575365 0.6672145 0.224379...,[ 0.17438498 -0.67538 -0.050255 -0.132791...,0.813332,820,26,0.510789,307


In [21]:
trial_data = validation_data[['qid','pid','NN_rank','score']]

In [22]:
trial_data = trial_data.reset_index(drop=True)

In [23]:
NN_dict = {}
qid_list = trial_data['qid'].unique()
for qid in qid_list:
    top_ones = trial_data[trial_data['qid'] == qid]
    top_ones = top_ones.reset_index(drop=True)
    top_ones = top_ones.sort_values(by=['NN_rank'])
    NN_dict[qid] = top_ones[:100]

In [24]:
f = open("NN.txt", "w")
for lr_df in NN_dict.values():
    for i, data in lr_df.iterrows():
        qid = str(data['qid'].astype(int))
        pid = str(data['pid'].astype(int))
        score = str(data['score'])
        rank = str(data['NN_rank'].astype(int))
        f.write(qid + "," + "A2" + "," + pid + "," + rank + "," + score + "," + "NN" + "\n")
f.close()

In [25]:
def average_precision_calc(df,retrieved,score,rank):
    average_precision = 0
    qid_list = np.unique(np.asarray(df['qid']))
    ranked_passages = df[df[rank] <= retrieved]

    relevant_passage = ranked_passages[ranked_passages['relevancy'] != 0]
    relevant_passage['rank'] = relevant_passage.groupby('qid')[score].rank(method = 'first',ascending=False)

    for qid in qid_list:
        temp = relevant_passage[relevant_passage['qid'] == qid]
        temp['rank'] = temp['rank']/temp[rank]
        if len(temp) == 0:
            average_precision += 0
        else:
            average_precision += sum(temp['rank'])/len(temp)

    average_precision = average_precision/len(qid_list)
    return average_precision

In [26]:
average_precision_calc(validation_data,100,'score','NN_rank')

0.03425544081728095

In [27]:
def NDCG_calc(df,retrieved, rank):

    all_DCG = 0
    relevant_passage = df[df['relevancy'] != 0]
    relevant_passage_retrived = relevant_passage[relevant_passage[rank] <= retrieved]

    qid_list = np.unique(np.asarray(df['qid']))

    for qid in qid_list:
        temp = relevant_passage[relevant_passage['qid'] == qid]
        DCG = sum(1/np.log2(np.asarray(temp[rank])+1))
        optDCG = sum(1/np.log2(np.arange(1,len(temp)+1)+1))
        all_DCG += DCG/optDCG
    all_DCG = all_DCG/len(qid_list)

    return all_DCG

In [28]:
NDCG_calc(validation_data,100,'NN_rank')

0.1638542308680274

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers, activations, losses, Model, Input
# from tensorflow.nn import leaky_relu
# import numpy as np
# from itertools import combinations
# from tensorflow.keras.utils import plot_model, Progbar
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from torch import nn

In [ ]:
# qids = final_train.qid.values
# doc_features = final_train[['cosine_similarity', 'DocLen', 'queryLen']].values
# doc_scores = final_train.relevancy.values

In [ ]:
# # put data into pairs
# xi = []
# xj = []
# pij = []
# pair_id = []
# pair_query_id = []
# for q in np.unique(qids):
#     query_idx = np.where(qids == q)[0]
#     for pair_idx in combinations(query_idx, 2):
#         pair_query_id.append(q)
        
#         pair_id.append(pair_idx)
#         i = pair_idx[0]
#         j = pair_idx[1]
#         xi.append(doc_features[i])
#         xj.append(doc_features[j])
        
#         if doc_scores[i] == doc_scores[j]:
#             _pij = 0.5
#         elif doc_scores[i] > doc_scores[j]:
#             _pij = 1
#         else: 
#             _pij = 0
#         pij.append(_pij)
        
# xi = np.array(xi)
# xj = np.array(xj)
# pij = np.array(pij)
# pair_query_id = np.array(pair_query_id)

# xi_train, xi_test, xj_train, xj_test, pij_train, pij_test, pair_id_train, pair_id_test = train_test_split(
#     xi, xj, pij, pair_id, test_size=0.2, stratify=pair_query_id)

In [ ]:
# qidsV = validation_data.qid.values
# doc_featuresV = validation_data[['cosine_similarity', 'DocLen', 'queryLen']]
# doc_scoresV = validation_data.relevancy.values

In [ ]:
# doc_featuresV.head()

In [ ]:
# xiV = []
# xjV = []
# # pijV = []
# # pair_idV = []
# # pair_query_idV = []
# uniqeu_qids = np.unique(qidsV)
# for i,q in enumerate(tqdm(uniqeu_qids[:1])):
#     query_idx = np.where(qidsV == q)[0]
#     for pair_idx in combinations(query_idx, 2):
#         # pair_query_idV.append(q)
        
#         # pair_idV.append(pair_idx)
#         i = pair_idx[0]
#         j = pair_idx[1]
#         xiV.append(doc_featuresV[i])
#         xjV.append(doc_featuresV[j])
        
#         # if doc_scoresV[i] == doc_scoresV[j]:
#         #     _pijV = 0.5
#         # elif doc_scoresV[i] > doc_scoresV[j]:
#         #     _pijV = 1
#         # else: 
#         #     _pijV = 0
#         # pijV.append(_pijV)
        
# xiV = np.array(xiV)
# xjV = np.array(xjV)
# # pair_query_idV = np.array(pair_query_idV)

In [ ]:
# class RankNet(Model):
#     def __init__(self):
#         super().__init__()
#         self.dense = [layers.Dense(16, activation=leaky_relu), layers.Dense(8, activation=leaky_relu)]
#         self.o = layers.Dense(1, activation='linear')
#         self.oi_minus_oj = layers.Subtract()
    
#     def call(self, inputs):
#         xi, xj = inputs
#         densei = self.dense[0](xi)
#         densej = self.dense[0](xj)
#         for dense in self.dense[1:]:
#             densei = dense(densei)
#             densej = dense(densej)
#         oi = self.o(densei)
#         oj= self.o(densej)
#         oij = self.oi_minus_oj([oi, oj])
#         output = layers.Activation('sigmoid')(oij)
#         return output

In [ ]:
# ranknet = RankNet()
# ranknet.compile(optimizer='adam', loss='binary_crossentropy')
# history = ranknet.fit([xi_train, xj_train], pij_train, epochs=10, batch_size=1000, validation_data=([xi_test, xj_test], pij_test))


In [ ]:
# ranknet.summary()

In [ ]:
# ranknet.call([xiV, xjV])

In [ ]:
# print(y_pred)